# 이 코드 설명
module_opt.py 모듈 내 sqp() 함수와 module_opt_AD.py 모듈 내 sqp() 함수 각각에 대해 잘 작동하는지 테스트하기 위해 작성한 스크립트.  
테스트 당시 테스트 상세 로그를 보기 위해 일시적으로 각 모듈 내 sqp() 함수에서의 내부(alm4sqp) 로그 print를 볼 수 있게 허용했다.  
그 후 다시 해당 로그를 볼 수 없게끔 조치했기에 본 스크립트는 추후 open 시 다시 돌리지 말고(절대 돌리지 마셈..다시 돌리면 sqp 로그밖에 안보이는 결과물들로 덮어씌워져서 원래 결과 날라감)  
아래 섹션들  
 - 1. module_opt.sqp() 함수 테스트
 - 2. module_opt_AD.sqp() 함수 테스트  

의 이미 기록된 결과 cell들을 보는용으로만 참고해라.  

특히 sqp 내부 iteration인 alm4sqp에서 어떻게  

p_k ... QP 부문제가 찾고자 하는 변수,  
Q_QPk ... QP 부문제의 목적함수,  
‖∇L_A‖ ... QP 부문제의 라그랑지안 함수의 변조 버전인 augmented lagrangian function의 기울기,  
‖ce_QPk‖∞ ... QP 부문제의 등호제약함수,  
‖ci_QPk‖∞ ... QP 부문제의 부등호제약함수,  
‖λ_ce_QPk‖, ‖ν_ci_QPk‖ ... QP 부문제의 라그랑지 승수  

값들이 변화하는지 참고하면 좋을 것이다.

### Initialization of Variables

In [1]:
all = [var for var in globals() if var[0] != '_']
for var in all:
    del globals()[var]
del all

### Module Import

In [2]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import sys
sys.path.append("../")
import importlib

import module_opt_AD
importlib.reload(module_opt_AD)
import module_opt_AD

import module_opt
importlib.reload(module_opt)
import module_opt

/home/cykwonlab/NumericalOptimizationCourse25/Assignment_3/../module_opt_AD.py:1138: SyntaxWarning: invalid escape sequence '\:'
  f (callable) : objective function(output \: single scalar)
/home/cykwonlab/NumericalOptimizationCourse25/Assignment_3/../module_opt.py:1215: SyntaxWarning: invalid escape sequence '\:'
  f (callable) : objective function(output \: single scalar)


### Test Functions for Constrained Optimization

1. Rosenbrock function constrained within a disk  
min &nbsp; $ f(x_1, x_2) = (1 - x_1)^2 + 100(x_2 - x_1^2)^2 $  
s.t. &nbsp; $ c(x_1, x_2) = 2 -x_1^2 - x_2^2 ≥ 0 $  
$ x*=(1.0, 1.0) $

2. Rosenbrock function on a disk  
min &nbsp; $ f(x_1, x_2) = (1 - x_1)^2 + 100(x_2 - x_1^2)^2 $  
s.t. &nbsp; $ c(x_1, x_2) = 1 - x_1^2 - x_2^2 = 0 $  
$ x*=(0.78642619, 0.61771572) $

3. Rosenbrock function on straight line  
min &nbsp; $ f(x_1, x_2) = (1 - x_1)^2 + 100(x_2 - x_1^2)^2 $  
s.t. &nbsp; $ c(x_1, x_2) = 1 - x_1 - x_2 = 0 $  
$ x*=(0.6188053, 0.3812165) $

4. Mishra's Bird function  
min &nbsp; $ f(x_1, x_2) = sin(x_2)e^{(1-cosx_1)^2} + cos(x_1)e^{(1-sinx_2)^2} + (x_1 - x_2)^2 $  
s.t. &nbsp; $ c(x_1, x_2) = - (x_1 + 5)^2 - (x_2 + 5)^2 + 25 ≥ 0 $  
$ x*=(-3.1302468, -1.5821422) $

5. Townsend function  
min &nbsp; $ f(x1, x_2) = -[cos{(x_1 - 0.1)x_2}]^2 - x_1sin(3x_1 + x_2) $  
s.t. &nbsp; $ c(x1, x_2) = -x_1^2 - x_2^2 + [2cos\{atan2(x_1, x_2)\} - \frac{1}{2}cos\{2atan2(x_1, x_2)\} - \frac{1}{4}cos\{3atan2(x_1, x_2)\} - \frac{1}{8}cos\{4atan2(x_1, x_2)\}]^2 + [2sin\{atan2(x_1, x_2)\}]^2 ≥ 0 $  
$ x*=(2.0052938, 1.1944509) $




In [3]:
### define functions
f_rosenbrock = lambda x : (1 - x[0])**2 + 100*(x[1] - x[0]**2)**2
f_rosenbrock_torch = lambda x : (1 - x[0])**2 + 100*(x[1] - x[0]**2)**2
c_rosenbrock = lambda x : 2 - x[0]**2 - x[1]**2
c_rosenbrock_torch = lambda x : 2 - x[0]**2 - x[1]**2

f_rosenbrock_eqdisk = lambda x : f_rosenbrock(x)
f_rosenbrock_eqdisk_torch = lambda x : f_rosenbrock_torch(x)
c_rosenbrock_eqdisk = lambda x : 1 - x[0]**2 - x[1]**2
c_rosenbrock_eqdisk_torch = lambda x : 1 - x[0]**2 - x[1]**2

f_rosenbrock_eqline = lambda x : f_rosenbrock(x)
f_rosenbrock_eqline_torch = lambda x : f_rosenbrock_torch(x)
c_rosenbrock_eqline = lambda x : 1 - x[0] - x[1]
c_rosenbrock_eqline_torch = lambda x : 1 - x[0] - x[1]

f_mishra = lambda x : np.sin(x[1])*np.exp((1-np.cos(x[0]))**2) + np.cos(x[0])*np.exp((1-np.sin(x[1]))**2) + (x[0] - x[1])**2
f_mishra_torch = lambda x : torch.sin(x[1])*torch.exp((1-torch.cos(x[0]))**2) + torch.cos(x[0])*torch.exp((1-torch.sin(x[1]))**2) + (x[0] - x[1])**2
c_mishra = lambda x : -(x[0] + 5)**2 - (x[1] + 5)**2 + 25
c_mishra_torch = lambda x : -(x[0] + 5)**2 - (x[1] + 5)**2 + 25

f_townsend = lambda x : -(np.cos((x[0] - .1)*x[1]))**2 - x[0]*np.sin(3*x[0] + x[1])
f_townsend_torch = lambda x : -(torch.cos((x[0] - .1)*x[1]))**2 - x[0]*torch.sin(3*x[0] + x[1])
c_townsend = lambda x : -x[0]**2 - x[1]**2 + (2*np.cos(np.atan2(x[0], x[1])) - .5*np.cos(2*np.atan2(x[0], x[1])) -.25*np.cos(3*np.atan2(x[0], x[1])) - .125*np.cos(4*np.atan2(x[0], x[1])))**2 + (2*np.sin(np.atan2(x[0], x[1])))**2
c_townsend_torch = lambda x : -x[0]**2 - x[1]**2 + (2*torch.cos(torch.atan2(x[0], x[1])) - .5*torch.cos(2*torch.atan2(x[0], x[1])) -.25*torch.cos(3*torch.atan2(x[0], x[1])) - .125*torch.cos(4*torch.atan2(x[0], x[1])))**2 + (2*torch.sin(torch.atan2(x[0], x[1])))**2


### 1. module_opt.sqp() 함수 테스트 - Finite Difference 기반

In [4]:
log = module_opt.sqp(f=f_rosenbrock, ce=[], ci=[c_rosenbrock], x0=np.array([-.5, -.5]), inner_opt=3, tol=1e-4, tol_inter=1e-3)


 log - ALM
‖∆p‖ = 2.14e+02, p01 = [152.99999999, 150.00000000] | Q_QPk = -2.2954e+04, ‖∇L‖ = 1.21e-08, ‖ce_QPk‖∞ = 0.00e+00, ‖ci_QPk‖∞ = 0.00e+00, ‖λ_ce_QPk‖ = 0.00e+00, ‖ν_ci_QPk‖ = 0.00e+00


 log - ALM
‖∆p‖ = 0.00e+00, p02 = [152.99999999, 150.00000000] | Q_QPk = -2.2954e+04, ‖∇L‖ = 1.21e-08, ‖ce_QPk‖∞ = 0.00e+00, ‖ci_QPk‖∞ = 0.00e+00, ‖λ_ce_QPk‖ = 0.00e+00, ‖ν_ci_QPk‖ = 0.00e+00

max_lm = 0.0
mu = 0.0
Dphi1_0 = -45908.99999748108
phi1_0 = 58.5
phi1_k = 53392212499.84484
phi1_k = 3250715849.468452
phi1_k = 192673100.92105997
phi1_k = 10800776.67880008
phi1_k = 536678.4818945418
phi1_k = 20009.256072346972
phi1_k = 300.3299237537851
phi1_k = 3.642875568998752

 log - SQP
‖∆x‖ = 1.67e+00, x01 = [0.69531250, 0.67187500] | f = 3.6429e+00, ‖∇L‖ = 6.50e+01, ‖ce‖∞ = 0.00e+00, ‖ci‖∞ = 0.00e+00, ‖λ‖ = 0.00e+00, ‖ν‖ = 0.00e+00 


 log - ALM
‖∆p‖ = 2.05e+02, p01 = [45.51612284, -24.64833372] | Q_QPk = -2.1100e+03, ‖∇L‖ = 6.13e-03, ‖ce_QPk‖∞ = 0.00e+00, ‖ci_QPk‖∞ = 2.91e+01, ‖λ_ce_QPk‖ = 0.00e

In [5]:
log = module_opt.sqp(f=f_rosenbrock_eqline, ce=[c_rosenbrock_eqline], ci=[], x0=np.array([-.5, -.5]), inner_opt=3, tol=1e-4, tol_inter=1e-3)


 log - ALM
‖∆p‖ = 7.24e+01, p01 = [52.66666666, 49.66666667] | Q_QPk = -1.2888e+04, ‖∇L‖ = 1.34e-08, ‖ce_QPk‖∞ = 1.00e+02, ‖ci_QPk‖∞ = 0.00e+00, ‖λ_ce_QPk‖ = 1.00e+02, ‖ν_ci_QPk‖ = 0.00e+00


 log - ALM
‖∆p‖ = 6.45e+01, p02 = [7.06060607, 4.06060606] | Q_QPk = -1.6562e+03, ‖∇L‖ = 1.22e-08, ‖ce_QPk‖∞ = 9.12e+00, ‖ci_QPk‖∞ = 0.00e+00, ‖λ_ce_QPk‖ = 1.46e+02, ‖ν_ci_QPk‖ = 0.00e+00


 log - ALM
‖∆p‖ = 6.32e+00, p03 = [2.58942366, -0.41057637] | Q_QPk = -3.3116e+02, ‖∇L‖ = 3.84e-08, ‖ce_QPk‖∞ = 1.79e-01, ‖ci_QPk‖∞ = 0.00e+00, ‖λ_ce_QPk‖ = 1.50e+02, ‖ν_ci_QPk‖ = 0.00e+00


 log - ALM
‖∆p‖ = 1.26e-01, p04 = [2.50035627, -0.49964373] | Q_QPk = -3.0436e+02, ‖∇L‖ = 2.52e-08, ‖ce_QPk‖∞ = 7.13e-04, ‖ci_QPk‖∞ = 0.00e+00, ‖λ_ce_QPk‖ = 1.50e+02, ‖ν_ci_QPk‖ = 0.00e+00


 log - ALM
‖∆p‖ = 5.02e-04, p05 = [2.50000140, -0.49999856] | Q_QPk = -3.0425e+02, ‖∇L‖ = 3.65e-08, ‖ce_QPk‖∞ = 2.84e-06, ‖ci_QPk‖∞ = 0.00e+00, ‖λ_ce_QPk‖ = 1.50e+02, ‖ν_ci_QPk‖ = 0.00e+00


 log - ALM
‖∆p‖ = 0.00e+00, p06 = [2.5000014

In [6]:
log = module_opt.sqp(f=f_rosenbrock_eqdisk, ce=[c_rosenbrock_eqdisk], ci=[], x0=np.array([0.7, 0.7]), inner_opt=3, tol=1e-4, tol_inter=1e-3)


 log - ALM
‖∆p‖ = 7.17e+01, p01 = [52.47398374, -48.92601626] | Q_QPk = -2.5982e+03, ‖∇L‖ = 3.63e-09, ‖ce_QPk‖∞ = 4.95e+00, ‖ci_QPk‖∞ = 0.00e+00, ‖λ_ce_QPk‖ = 4.95e+00, ‖ν_ci_QPk‖ = 0.00e+00


 log - ALM
‖∆p‖ = 2.38e+00, p02 = [50.79291184, -50.60708817] | Q_QPk = -2.5721e+03, ‖∇L‖ = 8.10e-09, ‖ce_QPk‖∞ = 2.40e-01, ‖ci_QPk‖∞ = 0.00e+00, ‖λ_ce_QPk‖ = 6.15e+00, ‖ν_ci_QPk‖ = 0.00e+00


 log - ALM
‖∆p‖ = 1.20e-01, p03 = [50.70800922, -50.69199080] | Q_QPk = -2.5706e+03, ‖∇L‖ = 2.22e-09, ‖ce_QPk‖∞ = 2.43e-03, ‖ci_QPk‖∞ = 0.00e+00, ‖λ_ce_QPk‖ = 6.21e+00, ‖ν_ci_QPk‖ = 0.00e+00


 log - ALM
‖∆p‖ = 1.22e-03, p04 = [50.70714462, -50.69285537] | Q_QPk = -2.5706e+03, ‖∇L‖ = 1.74e-08, ‖ce_QPk‖∞ = 4.94e-06, ‖ci_QPk‖∞ = 0.00e+00, ‖λ_ce_QPk‖ = 6.21e+00, ‖ν_ci_QPk‖ = 0.00e+00


 log - ALM
‖∆p‖ = 2.49e-06, p05 = [50.70714286, -50.69285714] | Q_QPk = -2.5706e+03, ‖∇L‖ = 1.21e-08, ‖ce_QPk‖∞ = 1.01e-08, ‖ci_QPk‖∞ = 0.00e+00, ‖λ_ce_QPk‖ = 6.21e+00, ‖ν_ci_QPk‖ = 0.00e+00


 log - ALM
‖∆p‖ = 0.00e+00, p06 = 

In [7]:
log = module_opt.sqp(f=f_mishra, ce=[], ci=[c_mishra], x0=np.array([-3.0, -3.0]), inner_opt=3, tol=1e-4, tol_inter=1e-3)


 log - ALM
‖∆p‖ = 4.60e+01, p01 = [-29.51902546, 35.32153622] | Q_QPk = -1.2036e+03, ‖∇L‖ = 1.82e-03, ‖ce_QPk‖∞ = 0.00e+00, ‖ci_QPk‖∞ = 6.21e+00, ‖λ_ce_QPk‖ = 0.00e+00, ‖ν_ci_QPk‖ = 6.21e+00


 log - ALM
‖∆p‖ = 1.09e+00, p02 = [-30.28926565, 34.54891643] | Q_QPk = -1.1646e+03, ‖∇L‖ = 1.29e-08, ‖ce_QPk‖∞ = 0.00e+00, ‖ci_QPk‖∞ = 3.86e-02, ‖λ_ce_QPk‖ = 0.00e+00, ‖ν_ci_QPk‖ = 6.40e+00


 log - ALM
‖∆p‖ = 6.82e-03, p03 = [-30.29408501, 34.54409706] | Q_QPk = -1.1644e+03, ‖∇L‖ = 2.95e-09, ‖ce_QPk‖∞ = 0.00e+00, ‖ci_QPk‖∞ = 4.82e-05, ‖λ_ce_QPk‖ = 0.00e+00, ‖ν_ci_QPk‖ = 6.40e+00


 log - ALM
‖∆p‖ = 8.51e-06, p04 = [-30.29409103, 34.54409104] | Q_QPk = -1.1644e+03, ‖∇L‖ = 1.01e-09, ‖ce_QPk‖∞ = 0.00e+00, ‖ci_QPk‖∞ = 6.01e-08, ‖λ_ce_QPk‖ = 0.00e+00, ‖ν_ci_QPk‖ = 6.40e+00


 log - ALM
‖∆p‖ = 0.00e+00, p05 = [-30.29409103, 34.54409104] | Q_QPk = -1.1644e+03, ‖∇L‖ = 6.01e-06, ‖ce_QPk‖∞ = 0.00e+00, ‖ci_QPk‖∞ = 6.01e-08, ‖λ_ce_QPk‖ = 0.00e+00, ‖ν_ci_QPk‖ = 6.40e+00

max_lm = 6.404264992809942
mu = 6.4

In [8]:
log = module_opt.sqp(f=f_townsend, ce=[], ci=[c_townsend], x0=np.array([1.8, 0.5]), inner_opt=3, tol=1e-4, tol_inter=1e-3)


 log - ALM
‖∆p‖ = 8.05e-01, p01 = [0.63451659, 0.49619288] | Q_QPk = -2.2935e+00, ‖∇L‖ = 1.68e-10, ‖ce_QPk‖∞ = 0.00e+00, ‖ci_QPk‖∞ = 8.67e-01, ‖λ_ce_QPk‖ = 0.00e+00, ‖ν_ci_QPk‖ = 8.67e-01


 log - ALM
‖∆p‖ = 2.10e-01, p02 = [0.42701527, 0.52654461] | Q_QPk = -1.5289e+00, ‖∇L‖ = 7.29e-10, ‖ce_QPk‖∞ = 0.00e+00, ‖ci_QPk‖∞ = 1.03e-02, ‖λ_ce_QPk‖ = 0.00e+00, ‖ν_ci_QPk‖ = 9.18e-01


 log - ALM
‖∆p‖ = 2.51e-03, p03 = [0.42453430, 0.52690751] | Q_QPk = -1.5195e+00, ‖∇L‖ = 8.65e-11, ‖ce_QPk‖∞ = 0.00e+00, ‖ci_QPk‖∞ = 2.46e-05, ‖λ_ce_QPk‖ = 0.00e+00, ‖ν_ci_QPk‖ = 9.19e-01


 log - ALM
‖∆p‖ = 6.00e-06, p04 = [0.42452836, 0.52690838] | Q_QPk = -1.5194e+00, ‖∇L‖ = 1.76e-11, ‖ce_QPk‖∞ = 0.00e+00, ‖ci_QPk‖∞ = 5.87e-08, ‖λ_ce_QPk‖ = 0.00e+00, ‖ν_ci_QPk‖ = 9.19e-01


 log - ALM
‖∆p‖ = 0.00e+00, p05 = [0.42452836, 0.52690838] | Q_QPk = -1.5194e+00, ‖∇L‖ = 5.93e-06, ‖ce_QPk‖∞ = 0.00e+00, ‖ci_QPk‖∞ = 5.87e-08, ‖λ_ce_QPk‖ = 0.00e+00, ‖ν_ci_QPk‖ = 9.19e-01

max_lm = 0.9188859991945257
mu = 0.918885999194525

### 2. module_opt_AD.sqp() 함수 테스트 - Torch의 Automatic Difference 기반

In [9]:
log = module_opt_AD.sqp(f=f_rosenbrock, f_torch=f_rosenbrock_torch, ce=[], ce_torch=[], ci=[c_rosenbrock], ci_torch=[c_rosenbrock_torch], x0=np.array([-.5, -.5]), inner_opt=3, tol=1e-4, tol_inter=1e-3)

max_lm = 0.0
mu = 0.0
Dphi1_0 = -45909.0
phi1_0 = 58.5
phi1_k = 53392212508.5
phi1_k = 3250715850.0
phi1_k = 192673100.953125
phi1_k = 10800776.680664062
phi1_k = 536678.4819946289
phi1_k = 20009.256076812744
phi1_k = 300.3299238681793
phi1_k = 3.6428755670785904

 log - SQP
‖∆x‖ = 1.67e+00, x01 = [0.69531250, 0.67187500] | f = 3.6429e+00, ‖∇L‖ = 6.50e+01, ‖ce‖∞ = 0.00e+00, ‖ci‖∞ = 0.00e+00, ‖λ‖ = 0.00e+00, ‖ν‖ = 0.00e+00 

max_lm = 98.2788923234935
mu = 98.2788923234935
Dphi1_0 = -407.2084427255674
phi1_0 = 3.6428755670785904
phi1_k = 113512.52539736535
phi1_k = 12626.258286750242
phi1_k = 1689.6399332952374
phi1_k = 226.68718885586816
phi1_k = 32.52578130578416
phi1_k = 2.8775417272039627

 log - SQP
‖∆x‖ = 1.97e-01, x02 = [0.84376023, 0.54301920] | f = 2.8775e+00, ‖∇L‖ = 2.34e+02, ‖ce‖∞ = 0.00e+00, ‖ci‖∞ = 0.00e+00, ‖λ‖ = 0.00e+00, ‖ν‖ = 9.83e+01 

max_lm = 0.0
mu = 0.0
Dphi1_0 = -6.338460521949057
phi1_0 = 2.8775417272039627
phi1_k = 0.24781060833572288

 log - SQP
‖∆x‖ = 3.02e-01,

In [10]:
log = module_opt_AD.sqp(f=f_rosenbrock_eqline, f_torch=f_rosenbrock_eqline_torch, ce=[c_rosenbrock_eqline], ce_torch=[c_rosenbrock_eqline_torch], ci=[], ci_torch=[], x0=np.array([-.5, -.5]), inner_opt=3, tol=1e-4, tol_inter=1e-3)

max_lm = 150.49999858059954
mu = 150.49999858059954
Dphi1_0 = -608.5008544790714
phi1_0 = 359.4999971611991
phi1_k = 2501.004688283537
phi1_k = 322.8280027542829

 log - SQP
‖∆x‖ = 1.27e+00, x01 = [0.75000071, -0.74999929] | f = 1.7233e+02, ‖∇L‖ = 5.55e+02, ‖ce‖∞ = 1.00e+00, ‖ci‖∞ = 0.00e+00, ‖λ‖ = 1.50e+02, ‖ν‖ = 0.00e+00 

max_lm = 149.08352889599598
mu = 149.08352889599598
Dphi1_0 = -1002.6509474134898
phi1_0 = 321.4115350802171
phi1_k = 128.6656440934964

 log - SQP
‖∆x‖ = 2.10e+00, x02 = [-0.15136033, 1.15136017] | f = 1.2867e+02, ‖∇L‖ = 4.32e+02, ‖ce‖∞ = 1.56e-07, ‖ci‖∞ = 0.00e+00, ‖λ‖ = 1.49e+02, ‖ν‖ = 0.00e+00 

max_lm = 130.36852779105112
mu = 130.36852779105112
Dphi1_0 = -46.84736016922558
phi1_0 = 128.6656411674711
phi1_k = 70.92490700805834

 log - SQP
‖∆x‖ = 4.15e-01, x03 = [0.14203767, 0.85796222] | f = 7.0925e+01, ‖∇L‖ = 1.83e+02, ‖ce‖∞ = 1.10e-07, ‖ci‖∞ = 0.00e+00, ‖λ‖ = 1.30e+02, ‖ν‖ = 0.00e+00 

max_lm = 331.03359160686244
mu = 331.03359160686244
Dphi1_0 = -432.122630

In [11]:
log = module_opt_AD.sqp(f=f_rosenbrock_eqdisk, f_torch=f_rosenbrock_eqdisk_torch, ce=[c_rosenbrock_eqdisk], ce_torch=[c_rosenbrock_eqdisk_torch], ci=[], ci_torch=[], x0=np.array([0.7, 0.7]), inner_opt=3, tol=1e-4, tol_inter=1e-3)

max_lm = 6.2091836709034265
mu = 6.2091836709034265
Dphi1_0 = -5141.228469512713
phi1_0 = 4.62418367341807
phi1_k = 725090894.7740113
phi1_k = 49490689.67167149
phi1_k = 3646872.4691184233
phi1_k = 304969.5993158922
phi1_k = 30544.626707396266
phi1_k = 3758.0240377682967
phi1_k = 545.7045807498255
phi1_k = 82.399659173417
phi1_k = 9.649283950249112
phi1_k = 0.1829588903336535

 log - SQP
‖∆x‖ = 1.40e-01, x01 = [0.79903739, 0.60099051] | f = 1.8079e-01, ‖∇L‖ = 2.15e+01, ‖ce‖∞ = 3.50e-04, ‖ci‖∞ = 0.00e+00, ‖λ‖ = 6.21e+00, ‖ν‖ = 0.00e+00 

max_lm = 0.27566720120417
mu = 0.27566720120417
Dphi1_0 = -0.2895649390042093
phi1_0 = 0.18088420816056153
phi1_k = 0.046350591746263714

 log - SQP
‖∆x‖ = 2.24e-02, x02 = [0.78569851, 0.61901585] | f = 4.6212e-02, ‖∇L‖ = 1.39e+00, ‖ce‖∞ = 5.03e-04, ‖ci‖∞ = 0.00e+00, ‖λ‖ = 2.76e-01, ‖ν‖ = 0.00e+00 

max_lm = 0.11785036903007293
mu = 0.11785036903007293
Dphi1_0 = -0.0011819762023644424
phi1_0 = 0.04627124557475142
phi1_k = 0.045674859002059344

 log - SQ

In [12]:
log = module_opt_AD.sqp(f=f_mishra, f_torch=f_mishra_torch, ce=[], ce_torch=[], ci=[c_mishra], ci_torch=[c_mishra_torch], x0=np.array([-3.0, -3.0]), inner_opt=3, tol=1e-4, tol_inter=1e-3)

max_lm = 6.404264993267233
mu = 6.404264993267233
Dphi1_0 = -2219.898682053611
phi1_0 = -11.043709832351485
phi1_k = 17723.610800602186
phi1_k = 4378.265316410436
phi1_k = 1016.3215721058929
phi1_k = 183.52720044203488
phi1_k = 18.747898118935794
phi1_k = -42.2159378176603

 log - SQP
‖∆x‖ = 1.44e+00, x01 = [-3.94669034, -1.92049716] | f = -4.2216e+01, ‖∇L‖ = 6.19e+01, ‖ce‖∞ = 0.00e+00, ‖ci‖∞ = 0.00e+00, ‖λ‖ = 0.00e+00, ‖ν‖ = 6.40e+00 

max_lm = 9.560842007202433
mu = 9.560842007202433
Dphi1_0 = -184.17584724879075
phi1_0 = -42.2159378176603
phi1_k = 53.81259935246035
phi1_k = -69.13713840391499

 log - SQP
‖∆x‖ = 1.17e+00, x02 = [-3.23606540, -0.99395404] | f = -6.9137e+01, ‖∇L‖ = 1.69e+02, ‖ce‖∞ = 0.00e+00, ‖ci‖∞ = 0.00e+00, ‖λ‖ = 0.00e+00, ‖ν‖ = 9.56e+00 

max_lm = 0.0
mu = 0.0
Dphi1_0 = -548.5597676863226
phi1_0 = -69.13713840391499
phi1_k = 94.43861751008993
phi1_k = 16.467803417686472
phi1_k = 12.161048385822767
phi1_k = -46.4259494337245
phi1_k = -75.64214956469472

 log - SQP
‖

In [13]:
log = module_opt_AD.sqp(f=f_townsend, f_torch=f_townsend_torch, ce=[], ce_torch=[], ci=[c_townsend], ci_torch=[c_townsend_torch], x0=np.array([1.8, 0.5]), inner_opt=3, tol=1e-4, tol_inter=1e-3)

max_lm = 0.9188859992900429
mu = 0.9188859992900429
Dphi1_0 = -1.7483597311938874
phi1_0 = 0.2374002438421871
phi1_k = -1.9021087570536888

 log - SQP
‖∆x‖ = 6.77e-01, x01 = [2.22452836, 1.02690838] | f = -2.5274e+00, ‖∇L‖ = 1.80e+00, ‖ce‖∞ = 0.00e+00, ‖ci‖∞ = 6.80e-01, ‖λ‖ = 0.00e+00, ‖ν‖ = 9.19e-01 

max_lm = 1.108150780795165
mu = 1.108150780795165
Dphi1_0 = -0.7051301895673738
phi1_0 = -1.7733165556011965
phi1_k = -2.0576500817128633

 log - SQP
‖∆x‖ = 3.26e-01, x02 = [2.01278178, 1.27542696] | f = -2.3092e+00, ‖∇L‖ = 7.96e-01, ‖ce‖∞ = 0.00e+00, ‖ci‖∞ = 2.27e-01, ‖λ‖ = 0.00e+00, ‖ν‖ = 1.11e+00 

max_lm = 1.1783506056774358
mu = 1.404833117415547
Dphi1_0 = -0.07614560562556409
phi1_0 = -1.990304279510501
phi1_k = -2.0158504179005976

 log - SQP
‖∆x‖ = 1.14e-01, x03 = [2.02949333, 1.16246985] | f = -2.0593e+00, ‖∇L‖ = 7.40e-01, ‖ce‖∞ = 0.00e+00, ‖ci‖∞ = 3.09e-02, ‖λ‖ = 0.00e+00, ‖ν‖ = 1.18e+00 

max_lm = 1.2983940564293786
mu = 1.2983940564293786
Dphi1_0 = -0.017700586979922255
phi1_